Most importantly, we will optimize to solve the fire as soon as possible, not maximize the time, so we will deploy the team with the lowest deploy time first (if available) in all cases. Also take into account the status of the fire to see how many units are needed, so that at the end we can calculate the operation cost = sum of all units that came.

Assume that the time that the firefighters arrive = start time + deployment time, and it will take another period = deployment time to go back and be ready.

Assume the price is per hour, and started counting when the firefighter arrive.

Assume that it takes 4 hours to deal with low severity, 12 hours to deal with medium severity and 36 hours for high severity

Assume that the efficiency point (ep) is equal to the price (in thousand) of each unit, so Tanker Planes(15) > Helicopters (8) > Smoke Jumpers (5) > Groud Crews (3) > Fire Engines (2), and multiple units can be used for the same mission, and to be able to solve the low: 4ep, medium: 12ep, and high: 36ep (similar to the hours)


Assume that for Flexible Units (Smoke Jumpers, Fire Engines, Ground Crews), the maximum we can deploy is 50% (just in case another fire happens), and for the bigger units (Helicopters and Tanker Planes) we can deploy all, since they should be only used for high severity and it doesnt happen a lot.

In [15]:
import pandas as pd
import json
import math
from datetime import datetime, timedelta

# File paths (update these paths as needed)
current_path = "C:\\Users\\taduc\\Downloads\\SAP-CONUHACKS-IX\\Data\\Preprocessed_1\\current_wildfiredata_cleaned.csv"
output_path = "C:\\Users\\taduc\\Downloads\\SAP-CONUHACKS-IX\\Data\\Output\\model1_logical.json"

# Load the CSV data
df = pd.read_csv(current_path)

# Convert the 'fire_start_time' column to datetime (we ignore the report timestamp per your note)
df['fire_start_time'] = pd.to_datetime(df['fire_start_time'])
df.head()


,timestamp,fire_start_time,severity,latitude,longitude
0,2024-01-01 00:00:00,2023-12-31 23:07:00,low,45.7747,-73.3052
1,2024-01-03 02:00:00,2024-01-03 01:53:00,low,44.5917,-72.7931
2,2024-01-03 10:00:00,2024-01-03 09:29:00,high,44.5963,-72.3298
3,2024-01-13 03:00:00,2024-01-13 02:04:00,medium,44.0596,-72.3365
4,2024-01-18 07:00:00,2024-01-18 06:54:00,low,45.8991,-72.4297


In [16]:
#2: Define firefighting resources and parameters.

# Each resource type is defined with:
#   - deployment_time: time required to travel to the fire (timedelta)
#   - cost: cost per hour (also represents the Efficiency Point (EP) in thousands)
#   - total: total units available
#   - ep: efficiency point per unit (equal to cost in thousands)
#   - flexible: if True, only up to 50% of units can be deployed for an event;
#               if False, all units may be used.
#   - available_times: a list tracking when each unit becomes available (initialized to datetime.min)
resources = {
    'Smoke Jumpers': {
        'deployment_time': timedelta(minutes=30),
        'cost': 5000,   # per hour
        'total': 5,
        'ep': 5,
        'flexible': True,
        'available_times': [datetime.min] * 5
    },
    'Fire Engines': {
        'deployment_time': timedelta(hours=1),
        'cost': 2000,
        'total': 10,
        'ep': 2,
        'flexible': True,
        'available_times': [datetime.min] * 10
    },
    'Helicopters': {
        'deployment_time': timedelta(minutes=45),
        'cost': 8000,
        'total': 3,
        'ep': 8,
        'flexible': False,
        'available_times': [datetime.min] * 3
    },
    'Tanker Planes': {
        'deployment_time': timedelta(hours=2),
        'cost': 15000,
        'total': 2,
        'ep': 15,
        'flexible': False,
        'available_times': [datetime.min] * 2
    },
    'Ground Crews': {
        'deployment_time': timedelta(hours=1, minutes=30),  # 1.5 hours
        'cost': 3000,
        'total': 8,
        'ep': 3,
        'flexible': True,
        'available_times': [datetime.min] * 8
    }
}

# Define the required EP and base fight duration (in hours) per severity level.
# For example, a low-severity fire requires 4 EP and normally takes 4 hours to deal with.
severity_requirements = {
    'low':    {'required_ep': 4,  'fight_duration': 4},
    'medium': {'required_ep': 12, 'fight_duration': 12},
    'high':   {'required_ep': 36, 'fight_duration': 36}
}

# Fixed damage costs for missed responses.
damage_costs = {
    'low': 50000,
    'medium': 100000,
    'high': 200000
}

# Initialize overall counters and cost accumulators.
total_operational_cost = 0
total_damage_cost = 0
addressed_count = 0
missed_count = 0

# For a detailed severity breakdown.
severity_report = {
    'low': {'addressed': 0, 'missed': 0},
    'medium': {'addressed': 0, 'missed': 0},
    'high': {'addressed': 0, 'missed': 0}
}

In [17]:
#3: Process each wildfire event.
# We sort the events by fire_start_time and (if simultaneous) by severity (with high first).
# For each event:
#   1. We determine the required EP and base fight duration.
#   2. We select available resource units (respecting the 50% limit for flexible resources)
#      ordering them by fastest deployment (and lowest cost as tie-breaker).
#   3. We "greedily" select units until the cumulative EP meets or exceeds the required EP.
#   4. We then compute the effective fight duration:
#         effective_fight_duration = base_fight_duration * (required_EP / deployed_EP)
#      so that if extra capacity is deployed, the fire is addressed faster.
#   5. For each deployed unit, we update its next available time and compute its cost.
#   6. We also build a schedule entry for the event.

# Sorting: if fires have the same fire_start_time, process higher severity first.
severity_sort_order = {'low': 1, 'medium': 2, 'high': 3}
df_sorted = df.sort_values(
    by=['fire_start_time', 'severity'],
    ascending=[True, False],
    key=lambda col: col.map(lambda x: severity_sort_order.get(x.lower().strip(), 0)) if col.name == 'severity' else col
)

# Initialize a list to record the schedule for each event.
schedule_list = []

# Process each fire event.
for idx, row in df_sorted.iterrows():
    # We use the fire_start_time as the event time.
    event_time = row['fire_start_time']
    sev = row['severity'].lower().strip()
    
    # Create a schedule entry for this event.
    event_schedule = {
        "event_id": idx,
        "fire_start_time": event_time.isoformat(),
        "severity": sev,
        "location": row.get("location", None),
        "assigned_units": [],
        "status": "addressed",  # Default; update to "missed" if no adequate team can be assembled.
        "event_operational_cost": 0
    }
    
    # Retrieve required EP and base fight duration (in hours) for this event.
    req_ep = severity_requirements[sev]['required_ep']
    base_fight_duration = severity_requirements[sev]['fight_duration']
    
    # Build candidate list: available units at event_time.
    # For each resource type, only consider up to 50% of units if the resource is flexible.
    candidates = []  # Each candidate is a dict with resource details.
    for res_type, info in resources.items():
        # Determine indices of units that are free at event_time.
        free_indices = [i for i, avail in enumerate(info['available_times']) if avail <= event_time]
        # For flexible units, limit the number of units to at most 50% of total.
        if info['flexible']:
            max_allowed = math.floor(info['total'] * 0.5)
            free_indices = free_indices[:max_allowed]
        # For each free unit, add its details as a candidate.
        for i in free_indices:
            candidates.append({
                'resource_type': res_type,
                'unit_index': i,
                'deployment_time': info['deployment_time'],  # timedelta
                'cost': info['cost'],                        # cost per hour
                'ep': info['ep']
            })
    
    # Sort candidates by deployment_time (fastest first) then by cost (lowest first).
    candidates.sort(key=lambda x: (x['deployment_time'], x['cost']))
    
    # Select candidates (in sorted order) until the cumulative EP meets or exceeds the required EP.
    selected = []
    total_deployed_ep = 0
    for candidate in candidates:
        selected.append(candidate)
        total_deployed_ep += candidate['ep']
        if total_deployed_ep >= req_ep:
            break
    
    # If we cannot meet the required EP, mark the event as "missed".
    if total_deployed_ep < req_ep:
        missed_count += 1
        total_damage_cost += damage_costs[sev]
        severity_report[sev]['missed'] += 1
        event_schedule["status"] = "missed"
        # No units are deployed; the schedule remains with an empty assigned_units list.
    else:
        addressed_count += 1
        severity_report[sev]['addressed'] += 1
        
        # Compute effective fight duration using the synergy effect:
        # If exactly the required EP is deployed, effective_fight_duration equals the base fight duration.
        # If extra EP is deployed, effective_fight_duration is proportionally lower.
        effective_fight_duration = base_fight_duration * (req_ep / total_deployed_ep)
        
        # Deploy the selected team.
        for candidate in selected:
            res_type = candidate['resource_type']
            unit_index = candidate['unit_index']
            dep_time = candidate['deployment_time']
            
            # Calculate arrival time: firefighters arrive at fire_start_time + deployment_time.
            arrival_time = event_time + dep_time
            
            # After arrival, they fight the fire for the effective fight duration.
            # Then they travel back (another deployment_time) before becoming available.
            busy_duration = dep_time + timedelta(hours=effective_fight_duration) + dep_time
            new_available_time = event_time + busy_duration
            
            # Update the resource unit's availability.
            resources[res_type]['available_times'][unit_index] = new_available_time
            
            # Operational cost is charged per hour starting from arrival.
            # We charge for the effective fight duration plus the return travel time (deployment_time in hours).
            dep_hours = dep_time.total_seconds() / 3600
            hours_charged = effective_fight_duration + dep_hours
            unit_cost = candidate['cost'] * hours_charged
            
            total_operational_cost += unit_cost
            event_schedule["event_operational_cost"] += unit_cost
            
            # Record the details for this deployed unit.
            event_schedule["assigned_units"].append({
                "resource_type": res_type,
                "unit_index": unit_index,
                "deployment_time_minutes": dep_time.total_seconds() / 60,
                "arrival_time": arrival_time.isoformat(),
                "new_available_time": new_available_time.isoformat(),
                "unit_cost": unit_cost
            })
    
    # Append this event's schedule to the overall schedule list.
    schedule_list.append(event_schedule)

In [18]:
#4: Compile the final report and include the schedule, then save as a JSON file.

report = {
    "number_of_fires_addressed": addressed_count,
    "number_of_fires_delayed": missed_count,
    "total_operational_costs": total_operational_cost,
    "estimated_damage_costs_from_delayed_responses": total_damage_cost,
    "fire_severity_report": severity_report,
    "schedule": schedule_list  # Detailed schedule for each fire event.
}

# Display summary report details.
print("Number of fires addressed:", report["number_of_fires_addressed"])
print("Number of fires delayed:", report["number_of_fires_delayed"])
print("Total operational costs: $", report["total_operational_costs"])
print("Estimated damage costs from delayed responses: $", report["estimated_damage_costs_from_delayed_responses"])
print("Fire severity report:", report["fire_severity_report"])

# Save the final report (including the schedule) to the specified JSON file.
with open(output_path, 'w') as f:
    json.dump(report, f, indent=4)

print("Report (with schedule) saved to", output_path)

Number of fires addressed: 32
Number of fires delayed: 0
Total operational costs: $ 11211000.0
Estimated damage costs from delayed responses: $ 0
Fire severity report: {'low': {'addressed': 14, 'missed': 0}, 'medium': {'addressed': 11, 'missed': 0}, 'high': {'addressed': 7, 'missed': 0}}
Report (with schedule) saved to C:\Users\taduc\Downloads\SAP-CONUHACKS-IX\Data\Output\model1_logical.json
